# NEON

This notebook is an excercise in executing ISOFIT on two dates from the NEON dataset and interpreting the outputs of ISOFIT. 

In [1]:
%matplotlib inline

In [2]:
# Setup logging
import logging
import os

logging.getLogger().setLevel(logging.INFO)

BASE = '/home/ray'

## Setup

The NEON dataset is structured as the following:
```
├── FieldSpectrometer
│   ├── BlackTarp01
│   │   ├── Data
│   │   │   ├── BlackTarp01_IndScienceSpectra.dat
│   │   │   ├── BlackTarp01_PanelBRDF.dat
│   │   │   ├── BlackTarp01_PanelSpectra.dat
│   │   │   ├── BlackTarp01_RadTrace.dat
│   │   │   ├── BlackTarp01_Refl.dat
│   │   │   ├── BlackTarp01_ScienceSpectra.dat
│   │   │   └── BlackTarp01_optNum.dat
│   │   └── Plot
│   │       ├── 01_BlackTarp01_optNum.png
│   │       ├── 02_BlackTarp01_RadTrace.png
│   │       ├── 03_BlackTarp01_PanelBRDF.png
│   │       ├── 04_BlackTarp01_PanelSpectra.png
│   │       ├── 05_BlackTarp01_ScienceSpectra.png
│   │       └── 06_BlackTarp01_Refl.png
│   ├── BlackTarp02
│   │   └── ... [trimmed]
│   └── ... [trimmed]
├── Radiance
│   ├── NIS01_20210403_173647_obs_ort
│   ├── NIS01_20210403_173647_obs_ort.hdr
│   ├── NIS01_20210403_173647_rdn_ort
│   ├── NIS01_20210403_173647_rdn_ort.hdr
│   ├── NIS01_20210403_174150_obs_ort
│   ├── NIS01_20210403_174150_obs_ort.hdr
│   ├── NIS01_20210403_174150_rdn_ort
│   └── NIS01_20210403_174150_rdn_ort.hdr
├── Reflectance
│   ├── NIS01_20210403_173647_rdn_ort_atm_nodata.bsq
│   ├── NIS01_20210403_173647_rdn_ort_atm_nodata.hdr
│   ├── NIS01_20210403_174150_rdn_ort_atm_nodata.bsq
│   └── NIS01_20210403_174150_rdn_ort_atm_nodata.hdr
└── Report
    └── 20210403_R10E_P1C2_TBMT_VicariousCalibration_Report.pdf
```

ISOFIT needs at minimum three pieces as input:

    1. Radiance measurements (rdn)
    2. Observation values    (obs)
    3. Location information  (loc)

NEON provides the RDN and OBS files for two dates:

```
Radiance
├── 173647
│   ├── NIS01_20210403_173647_obs_ort
│   ├── NIS01_20210403_173647_obs_ort.hdr
│   ├── NIS01_20210403_173647_rdn_ort
│   └── NIS01_20210403_173647_rdn_ort.hdr
└── 174150
    ├── NIS01_20210403_174150_obs_ort
    ├── NIS01_20210403_174150_obs_ort.hdr
    ├── NIS01_20210403_174150_rdn_ort
    └── NIS01_20210403_174150_rdn_ort.hdr
```

As such, the LOC file needs to be generated or faked. 


In [3]:
import numpy as np

# Extract the image locations of each point of interest (POI)
# These are defined in the NEON report as pixel locations, so we round here to convert to indices
report = {}
report['173647'] = {           # Upp L Y  | Low R Y  | Upp L X | Low R X
    'WhiteTarp': np.round([2224.9626, 2230.9771, 316.0078, 324.9385,]).astype(int),
    'BlackTarp': np.round([2224.9626, 2231.0032, 328.0086, 333.9731,]).astype(int),
    'Veg'      : np.round([2245.0381, 2258.8103, 343.9006, 346.9423,]).astype(int),
    'EWRoad'   : np.round([2214.9905, 2216.9978, 348.9902, 373.0080,]).astype(int),
    'NSRoad'   : np.round([2205.9580, 2225.9612, 357.9536, 359.9608,]).astype(int)
}
report['174150'] = {           # Upp L Y | Low R Y | Upp L X  | Low R X
    'WhiteTarp': np.round([653.9626, 659.9771, 3143.0078, 3151.9385]).astype(int),
    'BlackTarp': np.round([653.9626, 660.0032, 3155.0086, 3160.9731]).astype(int),
    'Veg'      : np.round([674.0381, 687.8103, 3170.9006, 3173.9423]).astype(int),
    'EWRoad'   : np.round([643.9905, 645.9978, 3175.9902, 3200.0080]).astype(int),
    'NSRoad'   : np.round([634.9580, 654.9612, 3184.9536, 3186.9608]).astype(int)
}
# Converts numpy array to comma-separated string for ISOFIT
toString = lambda array: ', '.join(str(v) for v in array)

In [4]:
# Which NEON date to process - change this to process a different date
neon_ids = ['173647', '174150']
neon_id  = neon_ids[0]

# Select the locations from the neon id -- roi == Regions of Interest
roi = report[neon_id]

# Path to the input NEON data
input_path = f'{BASE}/isofit/.idea/neon/data/TableMountainCalibration/Radiance'

In [5]:
from spectral.io import envi

def getMetadata(file, remove=['fwhm', 'band names', 'wavelength', 'wavelength units']):
    rdn_ds   = envi.open(file)
    metadata = rdn_ds.metadata.copy()
    for key in remove:
        if key in metadata:
            del metadata[key]
        else:
            print(f'Key {key!r} not found in the metadata, skipping')

    return metadata

def fakeLOC(rdn, lon, lat, elv, output=None, **kwargs):
    """
    """
    if not output:
        if 'rdn' in rdn:
            output = rdn.replace('rdn', 'loc')
        else:
            print('Error: No ouput file specified and cannot generate a unique name')
            return False

    metadata = getMetadata(rdn, **kwargs)
    metadata['bands'] = 3
    
    loc_ds = envi.create_image(output, metadata, ext='',force=True)
    loc    = loc_ds.open_memmap(interleave='bip', writable=True)

    loc[..., 0] = lon
    loc[..., 1] = lat
    loc[..., 2] = elv

    del loc
    del loc_ds

In [ ]:
fakeLOC(
    rdn = f'{input_path}/NIS01_20210403_{neon_id}_rdn_ort.hdr',
    lon = -105.237000,
    lat = 40.125000,
    elv = 1.689
)

# Apply OE

The next part walks through running the ISOFIT utility script `isofit/utils/apply_oe.py`. This is the first step of executing ISOFIT and will generate a default configuration.

In [ ]:
# If running inside the Docker, this is already set. You may need to point this to your sRTMnet installation path.
os.environ['EMULATOR_PATH'] # = "/path/to/sRTMnet"
os.environ['SIXS_DIR'] # = "/home/ray/6sv-2.1"

In [6]:
# ISOFIT output directory
output_path = f'{BASE}/tutorials/NEON/output/'

# NEON doesn't provide a surface file, generate one
f"{BASE}/isofit/examples/"

# Args for apply_oe since it's not intended to be called as a module
class ARGS:
    # NEON-specific arguments
    sensor            = f'neon'
    emulator_base     = os.environ['EMULATOR_PATH']
    logging_level     = 'INFO'
    n_cores           = os.cpu_count()
    segmentation_size = 40
    surface_category  = 'multicomponent_surface'
    presolve          = 0
    # int params can't get around config type checks
    empirical_line     = False
    analytical_line    = False
    segmentation_size  = 40
    pressure_elevation = 0
    # Do not change this
    ray_temp_dir = '/tmp/ray'
    
    def __init__(self, output, id):
        self.input_radiance    = f'{input_path}/NIS01_20210403_{id}_rdn_ort'
        self.input_loc         = f'{input_path}/NIS01_20210403_{id}_loc_ort'
        self.input_obs         = f'{input_path}/NIS01_20210403_{id}_obs_ort'
        self.working_directory = output
    
    # Allows apply_oe to not raise an exception on missing parameters
    def __getattr__(self, key):
        parent = super()
        if hasattr(parent, key):
            return getattr(parent, key)

args = ARGS(output_path, neon_id)

In [ ]:
# Generate the initial configuration
from isofit.utils.apply_oe import apply_oe

# This should take some time. If it doesn't, you should remove the try/except and rerun to see if a FileNotFoundError is being raised that isn't the surface.mat. 
try:
    apply_oe(args)
except FileNotFoundError:
    print('Surface.mat file missing, run the next code block to generate it')

## Surface Generation

The next part is to generate a `surface.mat` file. These files are uniquely generated based off of the `wavelengths.txt` file generated in the previous step by `apply_oe.py`. A `surface.json` is already provided for this example, but this is a config file that must be developed by hand. Examples you could copy are found in the `examples/20171108_Pasadena` and `examples/20151026_SantaMonica` of the ISOFIT repository. 

In [ ]:
from isofit.utils import surface_model

# Build the surface model
surface_model(f'{output_path}/config/surface.json')

print('Done')

## Running Retrievals

Now that `apply_oe` generated initial configurations and LUTs, and `surface_model` generated the `surface.mat`, now `Isofit` itself can be called to begin performing retrievals. Since everything is default, the results are not expected to be too great. 

In [ ]:
from isofit.core.isofit import Isofit

# Now run actual retrievals
model = Isofit(f'{output_path}/config/NIS01_20210403_{neon_id}_modtran.json')
# model = Isofit(f'{output_path}/config/NIS01_20210403_{neon_id}_h2o.json')

# model.run()

In [ ]:
region = 'Veg'
model.run(toString(roi[region]))

## Analyzing

Isofit has been successfully ran and a reflectance retrieval has been generated! 

---

Everything below here is still very much a work in progress

In [ ]:
# xarray and rioxarry (engine="rasterio") are not installed by default
import xarray as xr

from matplotlib import pyplot as plt

In [ ]:
# Open the output reflectance in xarray
ds = xr.open_dataset("/home/ray/tutorials/NEON/output/output/NIS01_20210403_173647_rfl", engine="rasterio")

In [ ]:
ds = xr.open_dataset("/home/ray/isofit/.idea/neon/data/TableMountainCalibration/Radiance/NIS01_20210403_173647_rdn_ort", engine="rasterio")

In [ ]:
# Retrieve the coordinates for this location
y0, y1, x0, x1 = coords = roi[region]

# Select only this area
ns = ds.isel(y=slice(y0, y1)).isel(x=slice(x0, x1))
ns = ns['band_data']
ns.load()

In [ ]:
mask = (1350 < ns.wavelength) & (ns.wavelength < 1450)
ns[mask, :] = np.nan
mask = (1800 < ns.wavelength) & (ns.wavelength < 1955)
ns[mask, :] = np.nan

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 5))

ns.mean(['x', 'y']).plot(x='wavelength', ax=ax)
ax.set_xticks(list(range(350, 2601, 250)))
ax.set_ylabel('Reflectance')
ax.set_title(f'Reflectance for {region} on {neon_id}')

In [ ]:
oxygen A-band feature about 765 nm -- big spike would mean geometry is wrong ie. elevation
CO2 is the big player of 2000s
typically shows as a "w" in about 1990 to 2100 range

## Iterating

The above highlighted some problem areas that need to be addressed. To get started, ISOFIT is comprised of three primary pieces:

1. The `Isofit` object
2. The `ForwardModel` object
3. The `IO` object

While the `Isofit` object is not required, unlike the other two, it is helpful to have the object on hand.

In [ ]:
from isofit.configs      import configs
from isofit.core.fileio  import IO
from isofit.core.forward import ForwardModel
from isofit.core.isofit  import Isofit
from isofit.inversion.inverse import Inversion

cfile  = f'{output_path}/config/NIS01_20210403_{neon_id}_modtran.json'
config = configs.create_new_config(cfile)

# First, load the required objects
im = Isofit(cfile) # 1, ISOFIT
fm = ForwardModel(config) # 2, Forward Model
io = IO(config, fm) # 3, IO object
iv = Inversion(config, fm)


In [ ]:
# Retrieve the coordinates for this location
region = 'Veg'
y0, y1, x0, x1 = coords = roi[region]

mid = lambda v0, v1: int(v0 + np.abs(v1 - v0) / 2)
indata = io.get_components_at_index(row=mid(y0, y1), col=mid(x0, x1))

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(io.meas_wl, indata.meas)
ax.set_xlabel('Wavelength')
ax.set_ylabel('Input data')

In [ ]:
states = iv.invert(indata.meas, indata.geom)

In [ ]:
def closest_wl(mv):
    return np.argmin(np.abs(io.meas_wl-mv))
wl_nan = io.meas_wl.copy()
wl_nan[closest_wl(1360):closest_wl(1410)] = np.nan
wl_nan[closest_wl(1800):closest_wl(1970)] = np.nan

In [ ]:
for n in range(states.shape[0]):
    plt.plot(wl_nan, states[n,:-2], label=f'{n}')
plt.legend()

In [ ]:
states[-1,-2:]

In [ ]:
indata.geom.__dict__